<a href="https://colab.research.google.com/github/mo3b1us/UFV_visualizacion/blob/main/ejercicios_clase/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COUNTRY**: país donde se ha producido la venta

**SUBRAND**: producto que se ha vendido

**YEAR**: año en el que se ha hecho la venta

**MOUNTH**: mes en el que se ha hecho (o predicho) la venta

**SCENARIO**:
- $\texttt{ai_forecast}$: venta que se ha predicho
- $\texttt{actual}$: venta real que se ha producido

**FORECAST**: mes en el que se ha hecho la predicción (mes = periodo - 1 ej: P02 finales de enero; PF es diciembre)

**FORECAST_YEAR**: año en el que se hizo la predicción

**AMOUNT**: volumen de ventas en litros

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv("datos_ejercicio_ventas.csv")
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


# Ejercicio 1

## Cada País

In [29]:
# Filtra solo los datos donde SCENARIO es "actual"
df_actual = df[df['SCENARIO'] == "actual"]

country_fig = px.histogram(df_actual, x='COUNTRY', y='AMOUNT',
                           title='Distribución de Ventas por País',
                           labels={'AMOUNT': 'Volumen de ventas (Litros)', 'COUNTRY': 'País'})
country_fig.show()

In [28]:
for country in df_actual["COUNTRY"].unique():
    sub_df = df_actual[df_actual["COUNTRY"] == country]
    subrand_pie_fig = px.pie(sub_df, names='SUBBRAND', values='AMOUNT',
                             title=f'Proporción de Ventas por Marca en {country}',
                             labels={'AMOUNT': 'Volumen de Ventas (Litros)', 'SUBBRAND': 'Marca'})
    subrand_pie_fig.show()

## Cada Mes y Año

In [30]:
df_actual['MONTH'] = pd.Categorical(df_actual['MONTH'], categories=range(1, 13), ordered=True)

monthly_sales = df_actual.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().unstack()

monthly_sales_fig = px.bar(monthly_sales.reset_index().melt(id_vars="YEAR", var_name="MONTH", value_name="SALES_VOLUME"),
             x="YEAR", y="SALES_VOLUME", color="MONTH", barmode="group",
             title="Distribución Mensual de las Ventas a lo largo de los Años",
             labels={'YEAR': 'Año', 'SALES_VOLUME': 'Volumen de Ventas (Litros)', 'MONTH': 'Mes'})

monthly_sales_fig.show()

<ipython-input-30-8a0c90c87358>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-30-8a0c90c87358>:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Cada Marca

In [31]:
subrand_hist_fig = px.histogram(df_actual, x='SUBBRAND', y='AMOUNT',
                            title='Distribución de Ventas por Marca',
                            labels={'AMOUNT': 'Volumen de Ventas (Litros)', 'SUBBRAND': 'Marca'})
subrand_hist_fig.show()

In [32]:
subrand_pie_fig = px.pie(df_actual, names = 'SUBBRAND', values = 'AMOUNT',
                            title='Proporción de Ventas por Marca',
                            labels={'AMOUNT': 'Volumen de Ventas (Litros)', 'SUBBRAND': 'Marca'},
                            color='SUBBRAND', color_discrete_sequence = px.colors.qualitative.G10)

subrand_pie_fig.show()

# Ejercicio 2

## País con Menos Ventas

In [17]:
# Agrupa por país y suma las ventas, luego selecciona el país con menores ventas
country_sales = df_actual.groupby('COUNTRY')['AMOUNT'].sum()
lowest_sales_country = country_sales.idxmin()

# Filtra el DataFrame para el país con las ventas más bajas y convierte 'YEAR' y 'MONTH' a una columna de fecha
lowest_country = df_actual[df_actual['COUNTRY'] == lowest_sales_country]
lowest_country['DATE'] = pd.to_datetime(lowest_country[['YEAR', 'MONTH']].assign(DAY=1))

# Agrupa por año y mes para calcular las ventas mensuales
monthly_sales_df = df_actual.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
monthly_sales_df['DATE'] = pd.to_datetime(monthly_sales_df[['YEAR', 'MONTH']].assign(DAY=1))

# Grafica la tendencia de ventas mensuales
trend_sales_fig = px.line(monthly_sales_df, x='DATE', y='AMOUNT',
                          title=f"Tendencia de las Ventas Mensuales de {lowest_sales_country}",
                          labels={'DATE': 'Fecha', 'AMOUNT': 'Volumen de Venta (Litros)'})

trend_sales_fig.show()

<ipython-input-17-df407655da36>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-17-df407655da36>:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [18]:
# 3 MESES: Análisis de estacionalidad
lowest_country_3_month = lowest_country.set_index('DATE').resample('3ME')['AMOUNT'].sum().reset_index()
m3_country_fig = px.line(lowest_country_3_month, x='DATE', y='AMOUNT',
                 title=f"Estacionalidad a 3 meses de {lowest_sales_country}",
                 labels={'DATE': 'Fecha', 'AMOUNT': 'Volumen de Ventas (Litros)'})
m3_country_fig.show()

# 6 MESES: Análisis de estacionalidad
lowest_country_6_month = lowest_country.set_index('DATE').resample('6ME')['AMOUNT'].sum().reset_index()
m6_country_fig = px.line(lowest_country_6_month, x='DATE', y='AMOUNT',
                 title=f"Estacionalidad a 6 meses de {lowest_sales_country}",
                 labels={'DATE': 'Fecha', 'AMOUNT': 'Volumen de Ventas (Litros)'})
m6_country_fig.show()

# 1 AÑO: Análisis de estacionalidad
lowest_country_1_year = lowest_country.set_index('DATE').resample('1YE')['AMOUNT'].sum().reset_index()
y1_country_fig = px.line(lowest_country_1_year, x='DATE', y='AMOUNT',
                 title=f"Estacionalidad a 1 año de {lowest_sales_country}",
                 labels={'DATE': 'Fecha', 'AMOUNT': 'Volumen de Ventas (Litros)'})
y1_country_fig.show()

## Producto con Más Ventas

In [21]:
top_subbrand = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().idxmax()
df_top_subbrand = df_actual[df_actual['SUBBRAND'] == top_subbrand]

monthly_sales_top_subbrand = df_top_subbrand.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
monthly_sales_top_subbrand['DATE'] = pd.to_datetime(monthly_sales_top_subbrand[['YEAR', 'MONTH']].assign(DAY=1))

trend_subrand_fig = px.line(monthly_sales_top_subbrand, x='DATE', y='AMOUNT',
              title=f"Tendencia del producto más vendido: {top_subbrand}",
              labels={'DATE': 'Fecha', 'AMOUNT': 'Volumen de Ventas (Litros)'})
trend_subrand_fig.show()

<ipython-input-21-05031c275ac3>:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [22]:
monthly_sales_top_subbrand.set_index('DATE', inplace=True)

# 3 MESES: Análisis de estacionalidad
top_subbrand_3_month = monthly_sales_top_subbrand['AMOUNT'].resample('3ME').sum().reset_index()
fig_3m = px.line(top_subbrand_3_month, x='DATE', y='AMOUNT',
                 title=f"Estacionalidad a 3 meses de {top_subbrand}",
                 labels={'DATE': 'Date', 'AMOUNT': 'Volumen de Ventas (Litros)'})
fig_3m.show()

# 6 MESES: Análisis de estacionalidad
top_subbrand_6_month = monthly_sales_top_subbrand['AMOUNT'].resample('6ME').sum().reset_index()
fig_6m = px.line(top_subbrand_6_month, x='DATE', y='AMOUNT',
                 title=f"Estacionalidad a 6 meses de {top_subbrand}",
                 labels={'DATE': 'Date', 'AMOUNT': 'Volumen de Ventas (Litros)'})
fig_6m.show()

# 1 AÑO: Análisis de estacionalidad
top_subbrand_1_year = monthly_sales_top_subbrand['AMOUNT'].resample('1YE').sum().reset_index()
fig_1y = px.line(top_subbrand_1_year, x='DATE', y='AMOUNT',
                 title=f"Estacionalidad a 1 año de {top_subbrand}",
                 labels={'DATE': 'Fecha', 'AMOUNT': 'Volumen de Ventas (Litros)'})
fig_1y.show()


# Ejercicio 3

In [23]:
spain = df[df['COUNTRY'] == 'Spain']

predicted_sales = spain[spain['SCENARIO'] == 'AI_forecast']
actual_sales = spain[spain['SCENARIO'] == 'actual']

merged_data = pd.merge(predicted_sales, actual_sales,
                       on=['YEAR', 'MONTH', 'SUBBRAND'],
                       suffixes=('_pred', '_actual'))
subbrands = merged_data['SUBBRAND'].unique()
merged_data['DATE'] = pd.to_datetime(merged_data[['YEAR', 'MONTH']].assign(DAY=1))


for subbrand in subbrands:
    subbrand_data = merged_data[merged_data['SUBBRAND'] == subbrand].sort_values(by='DATE')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=subbrand_data['DATE'], y=subbrand_data['AMOUNT_pred'],
                             mode='lines+markers', name='Predicción', line=dict(dash='dash', color='orange')))
    fig.add_trace(go.Scatter(x=subbrand_data['DATE'], y=subbrand_data['AMOUNT_actual'],
                             mode='lines+markers', name='Venta Real', line=dict(color='red')))

    fig.update_layout(
        title=f"Prediciones vs Ventas Reales de {subbrand} en España",
        xaxis_title="Fecha",
        yaxis_title="Volumen de Venta (Litros)",
        legend_title="Leyenda"
    )

    fig.show()

In [24]:
merged_data['ABSOLUTE_ERROR'] = abs(merged_data['AMOUNT_pred'] - merged_data['AMOUNT_actual'])
merged_data['PERCENTAGE_ERROR'] = (merged_data['ABSOLUTE_ERROR'] / merged_data['AMOUNT_actual']) * 100
merged_data.head()

error_metrics = merged_data.groupby('SUBBRAND').agg(
    mean_absolute_error=('ABSOLUTE_ERROR', 'mean'),
    mean_percentage_error=('PERCENTAGE_ERROR', 'mean')
).reset_index()

# Error Medio Absoluto
mae_fig = px.bar(error_metrics, x='SUBBRAND', y='mean_absolute_error',
                 title="Error Medio Absoluto por Producto",
                 labels={'SUBBRAND': 'Producto', 'mean_absolute_error': 'Error Medio Absoluto (Litros)'})
mae_fig.show()

# Error Porcentual Medio
mpe_fig = px.bar(error_metrics, x='SUBBRAND', y='mean_percentage_error',
                 title="Error Porcentual Medio por Producto",
                 labels={'SUBBRAND': 'Producto', 'mean_percentage_error': 'Error Porcentual Medio (%)'})
mpe_fig.show()